# Anomaly detection with an Autoencoder
In this laboratory, you will train an autoencoder with a dataset of benign traffic to make it learn the profile of the normal behaviour of the network. The current script will test the trained autoencoder on the test set of benign traffic, so that you can understand the maximum error produced by the trained autoencoder in reconstructing the normal traffic. 

This operation gives you the idea of what threshold can be set to detect anomalies (all the traffic flows whose reconstruction error is higher than the treshold are classified as anomalies). Copy the last cell of this notebook to test the autoencoder on the DOS2019 dataset of DDoS attacks.  

In [ ]:
# Author: Roberto Doriguzzi-Corin
# Project: Lecture on Intrusion Detection with Deep Learning
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#   http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

import sys
import time
import glob
import pprint
import argparse

import keras.callbacks
import tensorflow as tf
import numpy as np
import random as rn
import os
import csv
import h5py
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

# Seed Random Numbers
SEED = 1
os.environ['PYTHONHASHSEED']=str(SEED)
np.random.seed(SEED)
rn.seed(SEED)
config = tf.compat.v1.ConfigProto(inter_op_parallelism_threads=1)

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dense,  Flatten, Conv2D, Reshape, Input, UpSampling2D
from tensorflow.keras.layers import  MaxPooling2D, GlobalMaxPooling2D
from tensorflow.keras.models import Model, Sequential, save_model, load_model, clone_model
from sklearn.metrics import f1_score, accuracy_score, confusion_matrix
from scipy.stats import *
from numpy.random import randint
import matplotlib.pyplot as plt

import tensorflow.keras.backend as K
tf.random.set_seed(SEED)
K.set_image_data_format('channels_last')
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
config.gpu_options.allow_growth = True  # dynamically grow the memory used on the GPU

In [ ]:
def mean_squared_error(x_true,x_pred):
    mse = ((x_true-x_pred)**2).mean(axis=(1,2))

    return mse

In [ ]:
def load_dataset(path):
    filename = glob.glob(path)[0]
    dataset = h5py.File(filename, "r")
    set_x_orig = np.array(dataset["set_x"][:])  # features
    set_y_orig = np.array(dataset["set_y"][:])  # labels

    X = np.reshape(set_x_orig, (set_x_orig.shape[0], set_x_orig.shape[1], set_x_orig.shape[2], 1))
    Y = set_y_orig

    return X, Y

## Autoencoder implementation
It's worth noting that you should use the same input and output shape for your autoencoder's input and output layer, otherwise you will receive a shape error.

In [ ]:
def AutoencoderMLP(input_shape=(10,11,1), learning_rate=0.01):
    model = Sequential(name="autoencoder-mlp")
    model.add(Flatten(input_shape=input_shape))
    model.add(Dense(256, activation='relu', name='enc0'))
    model.add(Dense(32, activation='relu', name='enc1'))
    model.add(Dense(8, activation='relu', name='enc2'))
    model.add(Dense(8, activation='relu', name='dec0'))
    model.add(Dense(32, activation='relu', name='dec1'))
    model.add(Dense(256, activation='relu', name='dec2'))
    model.add(Dense(input_shape[0] * input_shape[1], activation='sigmoid', name='dec3'))
    model.add(Reshape(input_shape))

    compileModel(model, learning_rate)
    print(model.summary())
    return model

In [ ]:
def compileModel(model,lr):
    optimizer = Adam(learning_rate=lr, beta_1=0.9, beta_2=0.999)
    model.compile(optimizer=optimizer, loss='mean_squared_error')  # here we specify the loss function

## Training phase
Here we train the autoencoder using a patience value of 10 epochs and a maximum numer of epochs set to 100. Note that, we do not need any labels in anomaly detection tasks, as the input and the expected output of an autoencoder are the same.

In [ ]:
X_train, _ = load_dataset("../Datasets/HTTP-Benign/*benign*" + '-train.hdf5')
X_val, _ = load_dataset("../Datasets/HTTP-Benign/*benign*" + '-val.hdf5')

autoencoder = AutoencoderMLP()
autoencoder.fit(X_train, X_train, batch_size=512, epochs=1000,validation_data=(X_val, X_val), callbacks=[EarlyStopping(monitor='val_loss',restore_best_weights=True, patience=25)])

print("Saving best model's weights...")
autoencoder.save("./" + autoencoder.name + ".h5")

## Preliminary test phase
The the trained autoencoder on the test set of benign traffic to verify the maximum error computed on the normal data. This information can be used to set the threshold for anomaly detection.

In [ ]:
dataset_filelist = glob.glob("../Datasets/HTTP-Benign/*test.hdf5")

model_file = glob.glob("." + "/*.h5")[0]
print ("Model: ", model_file)
model = load_model(model_file)

for dataset in dataset_filelist:
    X_test, _ = load_dataset(dataset)
    if "benign" in dataset:
        # here we test the model on the test set of benign traffic
        # Calculate the MSE loss between the original test data and the reconstructed test data
        X_decoded = model.predict(X_test, batch_size=2048)
        mse_loss = mean_squared_error(np.squeeze(X_test), np.squeeze(X_decoded))

        print("MSE loss on test data:", mse_loss)
        n, bins, patches = plt.hist(mse_loss,bins=100)
        plt.show()